<a href="https://colab.research.google.com/github/patzaa/Pytorch-ML/blob/main/Computer_Vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Computer Vision Skeleton

Classifing clothes using supervised learning on FashionMNIST Dataset:

*   ```Model Class``` 
*   ```Train Loop```
*   ```Test Loop```



In [11]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

class FashionCNN(nn.Module):
    
    def __init__(self):
        super(FashionCNN, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc1 = nn.Linear(in_features=64*6*6, out_features=600)
        self.drop = nn.Dropout2d(0.25)
        self.fc2 = nn.Linear(in_features=600, out_features=120)
        self.fc3 = nn.Linear(in_features=120, out_features=10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.drop(out)
        out = self.fc2(out)
        out = self.fc3(out)
        
        return out

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)


train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)


In [12]:
def training_loop(dataloader, model, loss_fn, optimizer): 

  size = len(dataloader.dataset)

  for batch, (x, y) in enumerate(dataloader):
    
    #compute prediction and loss
    pred = model(x)
    loss = loss_fn(pred,y)

    # Backprop
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(),  batch * len(x)
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def eval_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [13]:
LR = 1e-3
EPOCH = 20

model = FeedForward()

#define loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)


for t in range(EPOCH):
    print(f"Epoch {t+1}\n-------------------------------")
    training_loop(train_dataloader, model, loss_fn, optimizer)
    eval_loop(test_dataloader, model, loss_fn)
print("Done training!")

Epoch 1
-------------------------------
loss: 2.305161  [    0/60000]
loss: 2.300931  [ 6400/60000]
loss: 2.289136  [12800/60000]
loss: 2.279658  [19200/60000]
loss: 2.287672  [25600/60000]
loss: 2.268106  [32000/60000]
loss: 2.279979  [38400/60000]
loss: 2.270305  [44800/60000]
loss: 2.268209  [51200/60000]
loss: 2.237894  [57600/60000]
Test Error: 
 Accuracy: 30.6%, Avg loss: 0.035235 

Epoch 2
-------------------------------
loss: 2.270222  [    0/60000]
loss: 2.274974  [ 6400/60000]
loss: 2.237896  [12800/60000]
loss: 2.216947  [19200/60000]
loss: 2.245202  [25600/60000]
loss: 2.207590  [32000/60000]
loss: 2.223502  [38400/60000]
loss: 2.211525  [44800/60000]
loss: 2.209713  [51200/60000]
loss: 2.133419  [57600/60000]
Test Error: 
 Accuracy: 31.9%, Avg loss: 0.033923 

Epoch 3
-------------------------------
loss: 2.223890  [    0/60000]
loss: 2.228690  [ 6400/60000]
loss: 2.159470  [12800/60000]
loss: 2.101949  [19200/60000]
loss: 2.168039  [25600/60000]
loss: 2.122133  [32000/600